# climatecasechart, process surya outputs

Merge layout, reading order and ocr into a dataframe.

In [ ]:
#@title Imports

import re
import os

import nltk
import json
import string
import openai
import pickle
import pymupdf
import requests
import xlsxwriter
import langdetect
import pandas as pd
from tqdm.cli import tqdm
from multiprocessing import Pool
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
from nltk.tokenize import TreebankWordTokenizer

# nltk.download('punkt_tab')

tqdm.pandas()

# os.chdir('/slow-data/unitednationsclimatehealth/')
os.chdir('/ipfs-storage/ipfs/herbert/unitednationsclimatehealth/')

# keywords = pd.read_excel(
#     'data/Physical and Mental Health keywords (21_3_25)-translations.xlsx',
#     skiprows=2, header=None, names=['keyword', 'x', 'group']
# ).drop('x', axis=1)

keywords = pd.read_excel(
    # 'data/Physical and Mental Health keywords (24_3_25)-translations.xlsx',
    'data/keywords/Physical and Mental Health keywords (30_3_25).xlsx',
    sheet_name='translations'
).set_index(['Group', 'Category'])


tqdm.pandas()

experiment_date = '2025-03-28-run-01' #@param {"type": "string"}

case_details = pd.read_pickle(f'data/{experiment_date}-case-details.p3')
words = pd.read_pickle(f'data/{experiment_date}-words.p3')

In [ ]:
#@title Detect languages

rerun = False #@param {type:"boolean"}

if rerun or 'file_languages' not in globals():
  file_texts = words.groupby(['slug', 'filename'])['text'].apply(' '.join)
  file_languages = file_texts.progress_apply(langdetect.detect)

assert len(set(file_languages) - set(keywords.columns)) == 0

file_languages.rename('language').value_counts().rename('#').reset_index([]).T

100%|██████████| 899/899 [00:18<00:00, 48.46it/s]


language,en,fr,de,es,nl,it,pl,et,sv,cs,pt,no,fi,da
#,600,116,101,23,20,12,6,5,4,4,3,3,1,1


In [ ]:
#@title Tokenization
#@markdown Takes 15-25', approx 12' more after the second progressbar

rerun = False #@param {type:"boolean"}

tqdm.pandas()

tokenizer = TreebankWordTokenizer().span_tokenize
# ps = PorterStemmer()
# stem = ps.stem

all_upper = re.compile('^[^a-z]+$')
def stem(x):
  """ 'stemming', not really though. Just keeping all-upper texts
  that way, because they are probably abbreviations like AIDS, and
  lowering the rest, to imply case-insensitive for non-abbreviations."""
  return x if all_upper.match(x) else x.lower()

keywords_tokens = keywords.map(
  lambda text: tuple(stem(text[a:b]) for a, b in tokenizer(text)))
# determine max lookahead for the actual articles
max_tokens = keywords_tokens.map(len).max().to_dict()

def tokenize(text):
  return pd.DataFrame([
    {'start': a, 'end': b, 'word': text[a:b], 'token': stem(text[a:b])}
    for a, b in tokenizer(text)
  ])

if rerun or not os.path.exists(f'data/{experiment_date}-tokens.p3'):
  with Pool(20) as pool:
    promises = words['text'].progress_apply(lambda x: pool.apply_async(tokenize, (x, )))
    tokens = promises.progress_apply(lambda x: x.get())

  tokens = pd.concat(list(tokens), keys=tokens.index, names=tokens.index.names)

  tokens['language'] = tokens.groupby(['slug', 'filename'], as_index=False).progress_apply(
    lambda x: pd.Series(file_languages.loc[x.name[0], x.name[1]], index=x.index)).reset_index(0, drop=True)

  tokens.to_pickle(f'data/{experiment_date}-tokens.p3')
else:
  tokens = pd.read_pickle(f'data/{experiment_date}-tokens.p3')

if rerun or not os.path.exists(f'data/{experiment_date}-tokens-lookahead.p3'):
  # The token, appended with max_stems (language specific) other tokens, to allow
  # multy-token keyword matching.
  tokens['lookahead'] = tokens.groupby(['slug', 'filename'], as_index=False).progress_apply(
    lambda f: pd.Series(list(zip(*(f['token'].shift(-i) for i in range(max_tokens[file_languages[f.name]])))), index=f.index)
  ).reset_index(0, drop=True)
  tokens.to_pickle(f'data/{experiment_date}-tokens-lookahead.p3')
else:
  tokens = pd.read_pickle(f'data/{experiment_date}-tokens-lookahead.p3')

tokens.index.names = list(tokens.index.names[:-1]) + ['token']

In [ ]:
#@title Matching on terms
rerun = True #@param {type:"boolean"}

# allow subsetting on slug and fn orthogonally (or even skip slug alltogether)
assert tokens[[]].reset_index(['slug', 'filename']).groupby('filename')['slug'].apply(set).str.len().max() <= 1

if rerun or 'matches_per_language' not in globals():
  matches_per_language = dict()

for language in keywords_tokens.columns:
  if language in matches_per_language:
    continue

  language_words = tokens[tokens['language'] == language]
  language_tokens = keywords_tokens[language]

  # Filter on the first stems, to speed up the rest with a shortlist
  firsts = set(language_tokens.str[0])
  short_list = language_words['token'].isin(firsts)

  tqdm.pandas(desc=language, leave=False, position=0)

  # extract the terms that match the documents
  matches_per_language[language] = (
    language_words[short_list].progress_apply(
      lambda x: {
        term_tokens for term_tokens in language_tokens
        if x['lookahead'][:len(term_tokens)] == term_tokens
      }, axis=1)
  )

matches_per_language_ = {k:v[v.str.len() > 0] for k, v in matches_per_language.items() if len(v) > 0 }
matches = pd.concat(list(matches_per_language_.values()))
matches = matches.apply(lambda x: max(x, key=len))
matches

slug                                                                                   filename                       page  position  line  word  token
hugues-falys-fian-greenpeace-ligue-des-droits-humains-v-totalenergies-the-farmer-case  20240301_54998_petition-2.pdf  13    7         0     0     15              (health,)
                                                                                                                            8         0     0     14       (air, pollution)
                                                                                                                      19    9         0     0     11          (well-being,)
                                                                                                                                                  14              (health,)
                                                                                                                      21    6         1     0     10            

In [706]:
#@title Extract text for each match

words['prefix length'] = words['text'].str.len() + 1 # +1 for ' '.join
words['prefix length'] = (
    words.groupby(
      ['slug', 'filename', 'page', 'position']
    )['prefix length'].cumsum() - words['prefix length'])

matches_ = pd.DataFrame({
  k: {
    'start': int(term_words['start'].iloc[0]),
    'end': int(term_words['end'].iloc[-1]),
    # position w.r.t. ' ' joined block instead of word
    'position start': int(first_word['prefix length'] + term_words['start'].iloc[0]),
    'position end': int(last_word['prefix length'] + term_words['end'].iloc[-1]),
    'text': ' '.join(term_words['word']),
    'language': file_languages.loc[k[0], k[1]],
    'keyword': keywords[keywords_stems[file_languages.loc[k[0], k[1]]] == v].iloc[0]['en'],
    'set': keywords[keywords_stems[file_languages.loc[k[0], k[1]]] == v].index[0][0],
    'type': keywords[keywords_stems[file_languages.loc[k[0], k[1]]] == v].index[0][1],
  }
  for k, v in tqdm(matches.items(), total=len(matches), position=0)
  for term_words in [tokens.iloc[tokens.index.get_loc(k):].iloc[:len(v)]] # alias
  for first_word in [words.loc[k[:-1]]] # alias
  for last_word in [words.loc[term_words.index[-1][:-1]]] # alias
}).T

matches_.index.names = list(tokens.index.names)
matches_

100%|██████████| 11396/11396 [00:19<00:00, 586.12it/s]


start  \
slug                                               filename                      page position line word token         
hugues-falys-fian-greenpeace-ligue-des-droits-h... 20240301_54998_petition-2.pdf 13   7        0    0    15       74   
                                                                                      8        0    0    14       81   
                                                                                 19   9        0    0    11       58   
                                                                                                         14       77   
                                                                                 21   6        1    0    10       48   
...                                                                                                              ...   
anton-foley-and-others-v-sweden-aurora-case        20220510_18245_na.pdf         4    2        7    0    11       68   
                                                                                      3        2    0    1        16   
                                                                                                         4        39   
                                                                                               3    0    8        52   
                                                                                      8        1    0    3        26   

                                                                                                               end  \
slug                                               filename                      page position line word token       
hugues-falys-fian-greenpeace-ligue-des-droits-h... 20240301_54998_petition-2.pdf 13   7        0    0    15     80   
                                                                                      8        0    0    14      9   
                                                                                 19   9        0    0    11     68   
                                                                                                         14     83   
                                                                                 21   6        1    0    10     54   
...                                                                                                             ..   
anton-foley-and-others-v-sweden-aurora-case        20220510_18245_na.pdf         4    2        7    0    11     73   
                                                                                      3        2    0    1      22   
                                                                                                         4      44   
                                                                                               3    0    8      58   
                                                                                      8        1    0    3      35   

                                                                                                               position start  \
slug                                               filename                      page position line word token                  
hugues-falys-fian-greenpeace-ligue-des-droits-h... 20240301_54998_petition-2.pdf 13   7        0    0    15                74   
                                                                                      8        0    0    14                81   
                                                                                 19   9        0    0    11                58   
                                                                                                         14                77   
                                                                                 21   6        1    0    10               136   
...                                                                                                             

In [ ]:
#@title Language statistics

n_matches = matches_.groupby(['slug', 'filename'])['keyword'].apply(lambda x: x.str.len().sum())

file_metadata = pd.DataFrame({
  'language': file_languages,
  '# matches': file_languages.index.to_series().apply(n_matches.get).fillna(0).astype(int),
  '# files': 1,
  '# files with matches': file_languages.index.to_series().apply(lambda x: x in n_matches).astype(int),
  'matches': matches_.groupby(['slug', 'filename'])['keyword'].apply(list),
  'texts': matches_.groupby(['slug', 'filename'])['text'].apply(list),
}).sort_values('matches')

language_metadata = file_metadata.groupby('language')[['# files', '# files with matches', '# matches']].sum().astype(int).sort_values('# files', ascending=False)
language_metadata['matches'] = file_metadata.groupby('language')['matches'].apply(lambda x: list({x for x in x.dropna() for x in x}))
language_metadata['texts'] = file_metadata.groupby('language')['texts'].apply(lambda x: list({x for x in x.dropna() for x in x}))
language_metadata

,# files,# files with matches,# matches,matches,texts
language,,,,,
en,600,347,57388,"[heat stress, allergic asthma, cholera, health...","[heat stress, allergic asthma, Shocking, chole..."
fr,116,95,14609,"[heat stress, cholera, health, Non-communicabl...","[Organisation mondiale de la santé, leishmanio..."
de,101,61,11760,"[heat stress, allergic asthma, cholera, health...","[Hitzestress, Mortalität, Wohlbefinden, Chemik..."
es,23,9,1801,"[mental, cholera, pandemic, health, illnesses,...","[mental, Malaria, asma, bienestar, ánimo, deng..."
nl,20,18,1887,"[dehydrate, heat stress, cholera, pandemic, he...","[sterfgevallen, eco-angst, dood, Salmonella, s..."
it,12,7,808,"[accident, pandemic, health, illnesses, psycho...","[tradire, batteri, rabbia, malattie, mortalità..."
pl,6,2,36,"[health, accident]","[zdrowie, wypadek]"
et,5,3,119,"[concentrate, well-being, air pollution]","[heaolu, õhusaaste, keskenduda]"
sv,4,3,803,"[dehydrate, mental, health, illnesses, nutriti...","[mental, fästingar, psykologisk, torka, sjukdo..."


In [788]:
#@title Expand paragraphs and prepare for annotation export

#@markdown with a minimal character window to other paragraph and potentially
#@markdown dilate-cluster them together.

rerun = False #@param {type:"boolean"}
rematch = False #@param {type:"boolean"}
window_size = 250 #@param {"type": "integer"}

if rerun or 'block_texts' not in globals():
  block_texts = words.groupby(['slug', 'filename', 'page', 'position']).agg({
    'text': ' '.join})

if rerun or rematch or 'matched' not in block_texts.columns:
  block_matches = matches_.groupby(['slug', 'filename', 'page', 'position']).agg({
    'text': list, 'keyword': list,
    'position start': list, 'position end': list,
    'set': list, 'type': list,
  }).rename({'text': 'keyword text', 'position start': 'start', 'position end': 'end'}, axis=1)

  block_texts['matched'] = False
  block_texts.loc[block_matches.index, 'matched'] = True
  block_texts['length'] = block_texts['text'].str.len()

  matched_ = block_texts.columns.get_loc('matched')

  for idx in tqdm(block_matches.index, position=0):
    loc = block_texts.index.get_loc(idx)

    # add whole paragraphs to front and back untill at least window_size chars
    prepended, i = 0, 1
    while prepended < window_size and block_texts.index[loc-i][:2] == idx[:2]:
      prepended += block_texts.iloc[loc-i]['length']
      block_texts.iloc[loc-i, matched_] = True
      i += 1

    appended, i = 0, 1
    while appended < window_size and block_texts.index[loc+i][:2] == idx[:2]:
      appended += block_texts.iloc[loc+i]['length']
      block_texts.iloc[loc+i, matched_] = True
      i += 1

  block_texts['block'] = block_texts.groupby(['slug', 'filename'], as_index=False).apply(
    lambda x: x['matched'] & ~x['matched'].shift(1, fill_value=False)).cumsum().reset_index(0, drop=True)

matched_block_texts = block_texts.query('matched').copy()

# offset character ranges to blocks
matched_block_texts['block offset'] = matched_block_texts['text'].str.len() + 2
matched_block_texts['block offset'] = matched_block_texts.groupby(['slug', 'filename', 'block'], as_index=False)['block offset'].cumsum() - matched_block_texts['block offset']

matched_block_texts[block_matches.columns] = block_matches
matched_block_texts[block_matches.columns] = matched_block_texts[block_matches.columns].map(lambda x: x if x==x else [])
matched_block_texts['start'] = matched_block_texts.apply(lambda x: [v + x['block offset'] for v in x['start']], axis=1)
matched_block_texts['end'] = matched_block_texts.apply(lambda x: [v + x['block offset'] for v in x['end']], axis=1)

# Group blocks to 1 row each
grouped_blocks = matched_block_texts.reset_index(['page']).groupby(['slug', 'filename', 'block']).agg({
  'page': set, 'text': '\n\n'.join,
  'keyword text': lambda x: [x for x in x for x in x],
  'keyword': lambda x: [x for x in x for x in x],
  'start': lambda x: [x for x in x for x in x],
  'end': lambda x: [x for x in x for x in x],
  'set': lambda x: [x for x in x for x in x],
  'type': lambda x: [x for x in x for x in x],
})

grouped_blocks_export = grouped_blocks.copy()
grouped_blocks_export['page'] = grouped_blocks_export['page'].apply(lambda x: ', '.join(map(str, x)))
grouped_blocks_export = grouped_blocks_export.set_index(['page'], append=True)[['keyword', 'keyword text', 'text', 'start', 'end', 'set', 'type']].swaplevel('page', 'block', axis=0).sort_index()
grouped_blocks.to_pickle(f'data/{experiment_date}-match-blocks.p3')
grouped_blocks_export

keyword  \
slug                                 filename                       page   block                                                      
11477                                20231011_11477_decision.pdf    1      1                                        [air pollution]   
                                                                           2                              [concentrate, well-being]   
                                                                           3      [air pollution, air pollution, air pollution, ...   
18416                                20230530_Case-No-312022_na.pdf 11     5                                               [health]   
                                                                    11, 12 6                   [disease, mortality, health, health]   
...                                                                                                                             ...   
zad-de-la-colline-holcim-switzerland 20220124_75405_decision.pdf    25, 26 4242                                      [guilt, guilt]   
                                     20220629_75405_decision.pdf    19, 20 4244                            [health, health, health]   
                                                                    21, 22 4245                [guilt, guilt, guilt, guilt, health]   
                                                                    23     4246                                             [guilt]   
                                                                    9, 10  4243                                          [shocking]   

                                                                                                                       keyword text  \
slug                                 filename                       page   block                                                      
11477                                20231011_11477_decision.pdf    1      1                                            [õhusaaste]   
                                                                           2                                   [keskenduda, heaolu]   
                                                                           3      [õhusaaste, õhusaaste, õhusaaste, õhusaaste, õ...   
18416                                20230530_Case-No-312022_na.pdf 11     5                                               [health]   
                                                                    11, 12 6                   [disease, mortality, health, health]   
...                                                                                                                             ...   
zad-de-la-colline-holcim-switzerland 20220124_75405_decision.pdf    25, 26 4242                          [culpabilité, culpabilité]   
                                     20220629_75405_decision.pdf    19, 20 4244                               [santé, santé, santé]   
                                                                    21, 22 4245   [culpabilité, culpabilité, culpabilité, culpab...   
                                                                    23     4246                                       [culpabilité]   
                                                                    9, 10  4243                                          [choquant]   

                                                                                                                               text  \
slug                                 filename                       page   block                                                      
11477                                20231011_11477_decision.pdf    1      1      9.7. Pariisi kliimaleppe ja riiklike arengudok...   
                                                                           2      18. Kliimamõju tuleb ehitusloa andmisel arvest...   
                                                                        

In [823]:
#@title Export ranked

# ranked w.r.t. keywords and how often they occurrence in matches, documents
# contained in the highest ranked sheet only if matched on multiple keywords.

rerun = True #@param {type:"boolean"}
destination = 'data/2025-03-30-except-exports/' #@param {"type": "string"}

!echo scp -O -r il-c03:`pwd`/'keyword-rank-sample.xlsx' .

def write_sheet(blocks):
  bold = workbook.add_format({"bold": True, 'valign': 'top', 'align': 'left', "text_wrap": True})
  kw_bold = workbook.add_format({"bold": True, 'valign': 'bottom', 'align': 'left', "text_wrap": True})
  kw_bold = workbook.add_format({"bold": True, 'valign': 'bottom', 'align': 'left', "text_wrap": True})

  highlighted_p = workbook.add_format({
      "bold": True, 'valign': 'top', 'align': 'left', "text_wrap": True,
      'font_color': '009CEF'})
  highlighted_m = workbook.add_format({
      "bold": True, 'valign': 'top', 'align': 'left', "text_wrap": True,
      'font_color': 'DC002D'})
  formats = {'Physical health': highlighted_p, 'Mental health': highlighted_m}
  cell = workbook.add_format({"text_wrap": True, 'valign': 'top', 'align': 'left'})

  def xlsx_highlight(sample, formats):
    done = 0
    text = sample['text']
    items = []
    for a, b, t in zip(sample['start'], sample['end'], sample['type']):
      before = text[done:a]
      highlight = text[a:b]
      if before != '':
        items.extend([before])
      if highlight != '':
        items.extend([formats[t], highlight])
      done = b
    if text[done:] != '':
      items.append(text[done:])
    return items

  slug_column = 0
  fn_column = 1
  pages_column = 2
  text_column = 3
  terms_column = 4
  hidden_text_column = 15

  worksheet.write_string(0, slug_column, "link", bold)
  worksheet.set_column(slug_column, slug_column, 4)
  worksheet.write_string(0, fn_column, "filename", bold)
  worksheet.set_column(fn_column, fn_column, 20)
  worksheet.write_string(0, pages_column, "pages", bold)
  worksheet.set_column(pages_column, pages_column, 5)
  worksheet.write_string(0, terms_column, "keyword", bold)
  worksheet.set_column(terms_column, terms_column, 20)
  worksheet.write_string(0, text_column, "text", bold)
  worksheet.set_column(text_column, text_column, 150)

  worksheet.write_string(0, text_column, "hidden text", bold)
  worksheet.set_column(hidden_text_column, hidden_text_column, 150)

  keywords = blocks['keyword text'].explode()

  row_num=1

  for slug, slug_keywords in keywords.groupby('slug'):
    if len(slug_keywords) > 1:
      worksheet.merge_range(row_num, slug_column, row_num + len(slug_keywords)-1, slug_column, "")
    worksheet.write_string(row_num, slug_column, slug, bold)
    for fn, file_keywords in slug_keywords.groupby('filename'):
      if len(file_keywords) > 1:
        worksheet.merge_range(row_num, fn_column, row_num + len(file_keywords)-1, fn_column, "")
      worksheet.write_string(row_num, fn_column, fn, bold)
      for pages, page_keywords in file_keywords.groupby('page'):
        if len(page_keywords) > 1:
          worksheet.merge_range(row_num, pages_column, row_num + len(page_keywords)-1, pages_column, "")
        worksheet.write_string(row_num, pages_column, pages, bold)
        for block, block_keywords in page_keywords.groupby('block'):
          row = blocks.loc[slug, fn, pages, block]
          terms = row['keyword text']
          if len(block_keywords) > 1:
            worksheet.merge_range(row_num, text_column, row_num + len(block_keywords)-1, text_column, "")
          worksheet.write_rich_string(row_num, text_column, *xlsx_highlight(row, formats), cell)
          worksheet.write_rich_string(row_num, hidden_text_column, *xlsx_highlight(row, formats), cell)

          for _, keyword in block_keywords.items():
            worksheet.write_string(row_num, terms_column, keyword, kw_bold)
            row_num+=1


cnt = matches_['keyword'].value_counts()
keyword_ranking = cnt.reset_index()['keyword'].reset_index().set_index('keyword')['index'].rename('rank')
filename_rank = matches_.groupby(['slug', 'filename'])['keyword'].apply(lambda x: keyword_ranking.loc[x].min()).sort_values().rename('rank')
rank_cnt = filename_rank.value_counts()

filename_samples = filename_rank.reset_index().groupby('rank').apply(
  lambda x: x.sample(rank_cnt[x.name])[['slug', 'filename']], include_groups=False)

workbook = xlsxwriter.Workbook(f"keyword-rank-sample.xlsx")
for rank, filenames in filename_samples.groupby('rank')['filename']:
  keyword = keyword_ranking[keyword_ranking==rank].index[0]
  worksheet = workbook.add_worksheet(f'{keyword} ({cnt[keyword]})')
  block = grouped_blocks_export[
      grouped_blocks_export.index.to_frame()['filename'].isin(filenames)]
  block = block[block['keyword text'].str.len() > 0]
  write_sheet(block)
workbook.close()

!echo scp -O -r il-c03:`pwd`/'keyword-rank-sample.xlsx' .

scp -O -r il-c03:/ipfs-storage/ipfs/herbert/unitednationsclimatehealth/keyword-rank-sample.xlsx .
scp -O -r il-c03:/ipfs-storage/ipfs/herbert/unitednationsclimatehealth/keyword-rank-sample.xlsx .
